# Generator

This notebook is used to generate training and testing data for the ANNs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from random import *
from scipy.linalg import expm

In [ ]:
#Parameters
dt=0.05
omega=1
gamma=1

#Time, trajectories and deltas
t0=0
tf=50000
nt=int((tf-t0)/dt)
t=np.linspace(t0,tf,nt)
Ntraj=100000
nu=100
nsets=int(Ntraj/nu)
print("Nu=",nu)
print("Sets=",nsets)

ndelta=201
Deltamin=0
Deltamax=10
Deltalist=np.linspace(Deltamin,Deltamax,ndelta)
dDelta=Deltalist[1]-Deltalist[0]

rho0=np.array([1,0,0,0],float)

In [ ]:
njumps=40
#Use the two following lines in case you want to generate histogram data
#num_bins=50
#val_max=80

#delta_full=np.zeros(Ntraj,float)
delta_test=np.zeros(Ntraj,float)
#histo_full=np.zeros([Ntraj,num_clases],float)
#histo_test=np.zeros([Ntraj,num_clases],float)
#delay_full=[]
delay_test=[]

for w in range(0,nsets): #Loop over sets
    
    delta=uniform(Deltamin,Deltamax)
    
    #Operators
    a=np.array([[0,1],[0,0]],float)
    ata=np.dot(np.transpose(a),a)
    H=np.array([[0,omega],[omega,delta]],float)
    Heff=H-1j*gamma/2*ata
    M=expm(-1j*dt*Heff) 
    
    for j in range(0,nu): #Loop over measurements
        delta_test[w*nu+j]=delta_test[w*nu+j]+int(delta/dDelta)
        jumps=[]
        
        ket=np.zeros([2,nt],complex)
        ket[0,0]=np.sqrt(rho0[0]) #Initial ket 
        ket[1,0]=np.sqrt(rho0[3]) 
        
        it=0
        i=0
        t_ant=0
        
        #Quantum Jumps method
        while it<njumps:
            dP=gamma*dt*np.dot(np.transpose(np.conjugate(ket[:,i])),np.dot(ata,ket[:,i]))
            r=random()
            
            if r<dP: #Jump
                ket[:,i+1]=np.dot(a,ket[:,i])
                ket[:,i+1]=ket[:,i+1]/np.linalg.norm(ket[:,i+1])
                jumps.append(t[i]-t_ant) 
                t_ant=t[i]
                i=i+1
                it=it+1  
                
            else:    #No jump
                ket[:,i+1]=np.dot(M,ket[:,i])
                ket[:,i+1]=ket[:,i+1]/np.linalg.norm(ket[:,i+1])
                i=i+1            
            
        #hist, lims_bins=np.histogram(jumps, bins=num_bins, range=(0,val_max), density=False)
        #histo_test[w*nu+j,:]=hist
        
        jumps=np.array(jumps,float)
        delay_test.append(jumps)
        
        
    print("Set",w+1)

delay_test=np.array(delay_test,float)

np.save("delay_test.npy",delay_test)
#np.save("histo_test.npy",histo_test)
np.save("delta_test.npy",delta_test)